In [2]:
# Import numerical computing libraries
import numpy as np

# Import 3D mesh processing library
import trimesh

# Import mathematical functions
from scipy import interpolate

# Import for plotting and visualization
import matplotlib.pyplot as plt

# Import for random number generation
import random

In [3]:
def generate_parametric_model(params):
    """
    Generates a 3D model based on input parameters.

    Args:
        params (dict): A dictionary containing model parameters.

    Returns:
        mesh (trimesh.Trimesh): The generated 3D model as a mesh object.
    """
    # Extract parameters
    shape_type = params.get("shape", "cube")
    size = params.get("size", [1.0, 1.0, 1.0])  # Default size
    detail = params.get("detail", None)  # Level of detail or complexity
    seed = params.get("seed", None)  # Seed for randomness

    # Set random seed for reproducibility
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    # Validate sizes are positive
    if any(s <= 0 for s in size):
        raise ValueError("Size dimensions must be positive numbers.")

    # Generate mesh based on shape type
    if shape_type == "cube":
        # Create a cube mesh
        mesh = trimesh.creation.box(extents=size)
    elif shape_type == "sphere":
        # Create a sphere mesh
        radius = min(size) / 2  # Use the smallest dimension
        subdivisions = detail
        mesh = trimesh.creation.icosphere(subdivisions=subdivisions, radius=radius)
    elif shape_type == "cylinder":
        # Create a cylinder mesh
        radius = min(size[0], size[1]) / 2  # Use smallest of x and y dimensions
        height = size[2]
        sections = detail
        mesh = trimesh.creation.cylinder(radius=radius, height=height, sections=sections)
    else:
        # Default to cube if shape is unrecognized
        mesh = trimesh.creation.box(extents=size)

    return mesh

In [4]:
def generate_random_models(num_models):
    """
    Generates a specified number of random 3D models.

    Args:
        num_models (int): The number of models to generate.

    Returns:
        models (list): A list of tuples containing mesh objects and their parameters.
    """
    models = []
    shape_types = ["cube", "sphere", "cylinder"]
    for i in range(num_models):
        shape = random.choice(shape_types)
        size = [random.uniform(0.5, 2.0) for _ in range(3)]

        # Set valid detail levels based on the shape
        if shape == "cube":
            detail = None  # Not applicable for cube
        elif shape == "sphere":
            detail = random.randint(1, 3)  # Subdivisions must be >= 1
        elif shape == "cylinder":
            detail = random.randint(3, 12)  # Sections must be >= 3

        params = {"shape": shape, "size": size, "detail": detail, "seed": i}  # Use loop index as seed for uniqueness
        mesh = generate_parametric_model(params)
        models.append((mesh, params))
    return models

In [5]:
def save_models_to_stl(models, output_dir="models"):
    """
    Saves the list of models to STL files.

    Args:
        models (list): A list of tuples containing mesh objects and their parameters.
        output_dir (str): The directory where STL files will be saved.
    """
    import os

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for idx, (mesh, params) in enumerate(models):
        # Construct filename based on parameters
        shape = params["shape"]
        size_str = "_".join([f"{s:.2f}" for s in params["size"]])
        detail = params["detail"]
        filename = f"{shape}_{size_str}_detail{detail}_model{idx}.stl"
        filepath = os.path.join(output_dir, filename)

        # Export mesh to STL
        mesh.export(filepath)

        print(f"Saved model {idx} to {filepath}")

In [6]:
def slice_models(models_dir="models", output_dir="sliced_layers"):
    """
    Slices the STL models into layers and saves the layer data.

    Args:
        models_dir (str): Directory containing STL models.
        output_dir (str): Directory where sliced layer data will be saved.
    """
    import os
    import subprocess

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # List all STL files in the models directory
    stl_files = [f for f in os.listdir(models_dir) if f.endswith(".stl")]

    for stl_file in stl_files:
        stl_path = os.path.join(models_dir, stl_file)
        output_path = os.path.join(output_dir, stl_file.replace(".stl", ".gcode"))

        # Path to CuraEngine executable (use raw string)
        cura_engine_path = r"C:\Program Files\UltiMaker Cura 5.9.0\CuraEngine.exe"

        # Command to slice the model using CuraEngine
        command = [
            cura_engine_path,
            "slice",
            "-v",
            "-l",
            stl_path,
            "-o",
            output_path,
            # Additional slicing parameters can be added here
        ]

        try:
            # Run the slicing command
            subprocess.run(command, check=True)
            print(f"Sliced {stl_file} to {output_path}")
        except subprocess.CalledProcessError as e:
            print(f"Error slicing {stl_file}: {e}")
        except FileNotFoundError:
            print(f"CuraEngine not found at path: {cura_engine_path}")
            print("Please check the path and ensure CuraEngine is installed.")
            break  # Exit the loop if CuraEngine is not found

In [7]:
def extract_layer_data(gcode_dir="sliced_layers", output_dir="layer_images"):
    """
    Parses G-code files to extract layer data and generate images.

    Args:
        gcode_dir (str): Directory containing G-code files.
        output_dir (str): Directory where layer images will be saved.
    """
    import os

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    gcode_files = [f for f in os.listdir(gcode_dir) if f.endswith(".gcode")]

    for gcode_file in gcode_files:
        gcode_path = os.path.join(gcode_dir, gcode_file)
        with open(gcode_path, "r") as file:
            lines = file.readlines()

        layers = []
        current_layer = []
        for line in lines:
            if line.startswith(";LAYER:"):
                if current_layer:
                    layers.append(current_layer)
                    current_layer = []
            elif line.startswith("G1"):
                current_layer.append(line.strip())

        # Process layers as needed
        # For example, save each layer's G-code commands to a separate file
        for idx, layer in enumerate(layers):
            layer_filename = f"{gcode_file.replace('.gcode', '')}_layer{idx}.txt"
            layer_filepath = os.path.join(output_dir, layer_filename)
            with open(layer_filepath, "w") as layer_file:
                layer_file.write("\n".join(layer))

            print(f"Extracted layer {idx} from {gcode_file} to {layer_filepath}")

In [8]:
def create_table_of_contents(models, output_file="table_of_contents.csv"):
    """
    Creates a CSV file summarizing all generated models.

    Args:
        models (list): A list of tuples containing mesh objects and their parameters.
        output_file (str): The CSV file to save the table of contents.
    """
    import csv

    # Define the header
    header = ["Model Index", "Shape", "Size X", "Size Y", "Size Z", "Detail", "Filename"]

    # Open the CSV file for writing
    with open(output_file, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)

        for idx, (mesh, params) in enumerate(models):
            shape = params["shape"]
            size_x, size_y, size_z = params["size"]
            detail = params["detail"]
            filename = f"{shape}_{size_x:.2f}_{size_y:.2f}_{size_z:.2f}_detail{detail}_model{idx}.stl"
            writer.writerow([idx, shape, f"{size_x:.2f}", f"{size_y:.2f}", f"{size_z:.2f}", detail, filename])

    print(f"Table of contents saved to {output_file}")

In [9]:
def main():
    num_models = 10  # Specify the number of models to generate

    # Step 1: Generate random models
    models = generate_random_models(num_models)

    # Step 2: Save models to STL files
    save_models_to_stl(models)

    # Step 3: Slice models into layers
    slice_models()

    # Step 4: Extract layer data (optional)
    extract_layer_data()

    # Step 5: Create a table of contents
    create_table_of_contents(models)


if __name__ == "__main__":
    main()

Saved model 0 to models\cylinder_1.39_1.98_1.85_detail11_model0.stl
Saved model 1 to models\sphere_1.64_1.13_0.89_detail3_model1.stl
Saved model 2 to models\cube_1.35_1.70_0.59_detailNone_model2.stl
Saved model 3 to models\cube_0.64_1.04_0.75_detailNone_model3.stl
Saved model 4 to models\cube_1.39_0.70_1.87_detailNone_model4.stl
Saved model 5 to models\cube_0.95_1.58_1.22_detailNone_model5.stl
Saved model 6 to models\cylinder_0.88_1.04_1.54_detail11_model6.stl
Saved model 7 to models\cylinder_1.73_1.23_0.89_detail3_model7.stl
Saved model 8 to models\sphere_1.92_1.09_0.57_detail3_model8.stl
Saved model 9 to models\cube_1.06_1.06_0.79_detailNone_model9.stl
Error slicing cube_0.64_1.04_0.75_detailNone_model3.stl: Command '['C:\\Program Files\\UltiMaker Cura 5.9.0\\CuraEngine.exe', 'slice', '-v', '-l', 'models\\cube_0.64_1.04_0.75_detailNone_model3.stl', '-o', 'sliced_layers\\cube_0.64_1.04_0.75_detailNone_model3.gcode']' returned non-zero exit status 2.
Error slicing cube_0.95_1.58_1.22_d